In [ ]:
TEST = False
SAMPLE_SIZE = 10000

Import libs

In [ ]:
# import libs
import glob, pylab, pandas as pd
import pydicom, numpy as np
import random
import json
import time
import copy
import pydicom
import torchvision
import sys
import math

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt
from matplotlib import patches, patheffects
from multiprocessing import Pool

from sklearn.model_selection import train_test_split
from torchvision import datasets, transforms
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torch.optim import lr_scheduler
from pathlib import Path
from tqdm import tqdm     

# from fastai.conv_learner import *
# from fastai.dataset import *
# from fastai.dataset import ImageClassifierData

In [ ]:
# see input dir
!ls ../input

In [ ]:
PATH = Path('../input')
# read training lables
train_bb_df = pd.read_csv(PATH/'stage_1_train_labels.csv')
train_bb_df.head()

In [ ]:
!ls {PATH}/'stage_1_train_images'| wc -l

row 4 shows, there is bouding box in image with given x & y. First 4 image don't have any bounding box
### check if duplicate bounding box are present for any patient

In [ ]:
train_bb_df['duplicate'] = train_bb_df.duplicated(['patientId'], keep=False)
# see data
train_bb_df[train_bb_df['duplicate']].head()

In [ ]:
len(train_bb_df)

In [ ]:
detailed_df = pd.read_csv(PATH/'stage_1_detailed_class_info.csv')
# detailed_df.head()
# merge two df
class_df = train_bb_df.merge(detailed_df, on="patientId")
# len(class_df) # -> 35875 it is way more than original df.
# class_df[class_df.duplicated()]
class_df.drop_duplicates(inplace=True)
class_df.head() # len is 28989. 
csv_df = class_df.filter(['patientId', 'Target'], )
# csv_df.head()

Loading all train images in memory causes memory overflow. Kernel will restart. 
We creating dataset to read images, then apply transformation.

In [ ]:
img_dir = PATH/'stage_1_train_images'
sample = list(img_dir.iterdir())

if TEST:
    sample = sample[:SAMPLE_SIZE]
train, test = train_test_split(sample)
batch_size=32
sz=224
tloc = Path('/tmp/pytorch')/str(sz)
tloc.mkdir(parents=True, exist_ok=True)

def scale_to(x, ratio, targ): 
    '''Calculate dimension of an image during scaling with aspect ratio'''
    return max(math.floor(x*ratio), targ)

def read_image_resize(sample, tsize=sz, tloc=tloc, ):
    tloc.mkdir(parents=True, exist_ok=True)
    for loc in tqdm(sample):
        img_arr = pydicom.read_file(loc.as_posix()).pixel_array
        img_arr = np.stack([img_arr] * 3, axis=2)
        img_arr = img_arr/img_arr.max()
        img_arr = (255*img_arr).clip(0, 255).astype(np.uint8)
        img_arr = Image.fromarray(img_arr)#.convert('RGB') # model expects 3 channel image
        r,c = img_arr.size
        ratio = tsize/min(r,c)
        sz = (scale_to(r, ratio, tsize), scale_to(c, ratio, tsize))
        img_arr = img_arr.resize(sz, Image.LINEAR)
        filepath = tloc / loc.name
#         np.save(filepath, img_arr, )
        img_arr.save((tloc/loc.stem).as_posix()+'.png')

# read_image_resize()
# try:
#     if len(list(tloc.iterdir())) < SAMPLE_SIZE:
#         read_image_resize()
# except FileNotFoundError:
#     read_image_resize()

def resize_image(loc, tsize=256, tloc=tloc):
    img_arr = pydicom.read_file(loc.as_posix()).pixel_array
    img_arr = np.stack([img_arr] * 3, axis=2)
    img_arr = img_arr/img_arr.max()
    img_arr = (255*img_arr).clip(0, 255).astype(np.uint8)
    img_arr = Image.fromarray(img_arr)#.convert('RGB') # model expects 3 channel image
    r,c = img_arr.size
    ratio = tsize/min(r,c)
    sz = (scale_to(r, ratio, tsize), scale_to(c, ratio, tsize))
    img_arr = img_arr.resize(sz, Image.LINEAR)
    filepath = tloc / loc.name
#         np.save(filepath, img_arr, )
    img_arr.save((tloc/loc.stem).as_posix()+'.png')
    
pool = Pool()                         # Create a multiprocessing Pool
pool.map(resize_image, tqdm(sample))

In [ ]:
!ls {tloc} | wc -l

In [ ]:
class CDataset(Dataset):
    def __init__(self, ds, img_dir, class_df, transform=None, ext=None): 
        self.ds = ds
        self.img_dir = img_dir
        self.class_df = class_df
        self.ext = ext or '.dcm'
        self.transform = transforms.Compose(transform) if transform else None
        
    def __len__(self): 
        return len(self.ds)
    
    def read_dicom_image(self, pid):
#         pid = pid +'.dcm.npy'
#         return Image.fromarray(np.load(tloc/pid))
        pid = pid +'.png'
        return Image.open(tloc/pid)
    
    def get1item(self, i):
        patientId = self.ds[i].name.split('.')[0]
        img = self.read_dicom_image(patientId)
        if self.transform:
            img = self.transform(img)
        kls = self.class_df[self.class_df['patientId'] == patientId]
        return img, kls.iloc[0].Target
#         row = kls.iloc[0]
#         dim = []
#         for i in ['x', 'y', 'width', 'height']:
#             try:
#                 val = str(int(getattr(row, i)))
#             except ValueError:
#                 val = '-'
#             dim.append(val)
#         return img,dim
    
    def __getitem__(self, idx):
        return self.get1item(idx)

In [ ]:

transform = [
#     transforms.RandomHorizontalFlip(), 
#     transforms.TenCrop(224), 
#     transforms.Lambda(lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops])),
    
    transforms.ToTensor(),
#   transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
]
train_ds = CDataset(train, tloc, class_df, transform=transform)
test_ds = CDataset(test, tloc, class_df, transform=transform)

train_dl = DataLoader(train_ds, batch_size=batch_size,)
test_dl = DataLoader(test_ds, batch_size=batch_size)

In [ ]:
x,y = next(iter(train_dl))

In [ ]:
use_gpu = torch.cuda.is_available()
dataloaders = {'train': train_dl, 'val':test_dl}

def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train(True)  # Set model to training mode
            else:
                model.train(False)  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            data_loader = dataloaders[phase]
            start = time.time()
            for data in tqdm(data_loader):
                inputs, labels = data
                if use_gpu:
                    inputs = Variable(inputs.cuda(),requires_grad=True)
                    labels = Variable(labels.cuda())
                else:
                    inputs, labels = Variable(inputs), Variable(labels)

                optimizer.zero_grad()
                outputs = model(inputs)
                _, preds = torch.max(outputs.data, 1)
                loss = criterion(outputs, labels)
                if phase == 'train':
                    loss.backward()
                    optimizer.step()
                running_loss += loss.data[0] * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(data_loader.dataset)
            epoch_acc = running_corrects / len(data_loader.dataset)
            epoch_time = time.time() - start
            tqdm.write('{} Loss: {:.4f} Acc: {:.4f} in {:.0f}m {:.0f}s'.format(
                phase, epoch_loss, epoch_acc, epoch_time // 60, epoch_time % 60))
#             deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
#                 best_model_wts = copy.deepcopy(model.state_dict())

        tqdm.write('\n')

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
#     model.load_state_dict(best_model_wts)
    return model

In [ ]:
device = torch.cuda.set_device(0)
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

class Densenet(nn.Module):
    def __init__(self, num_classes = 2):
        super(Densenet,self).__init__()
        original_model = torchvision.models.densenet121()
        self.features = nn.Sequential(*list(original_model.children())[:-1])
        num_ftrs = original_model.classifier.in_features
        self.classifier = (nn.Linear(num_ftrs, num_classes))

    def forward(self, x):
        f = self.features(x)
        f = F.relu(f, inplace=True)
        f = F.avg_pool2d(f, kernel_size=7).view(f.size(0), -1)
        y = self.classifier(f)
        return y
    
model = Densenet()
# model = torchvision.models.densenet121(pretrained=True)
# # Freeze training for all layers
# for param in model.parameters():
#     param.require_grad = False
    
model.cuda()
# net = torch.nn.DataParallel(net, device_ids=range(torch.cuda.device_count()))
cudnn.benchmark = True
    
num_ftrs = model.classifier.in_features
print("num_ftrs : ", num_ftrs)
model.fc = nn.Linear(num_ftrs, 2)
criterion = nn.CrossEntropyLoss()#.cuda()

# Observe that all parameters are being optimized
optimizer_ft = optim.Adam(model.parameters())

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)
since = time.time()

Lets train our model and check classification accuracy

In [ ]:
model = train_model(model, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=10)

In [ ]:
! nvidia-smi 